**Predict TripAdvisor Rating**

*Проект № 3. О вкусной и здоровой пище*

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

In [ ]:
# изменим название одного из городов 'Oporto' на более используемое в справочниках 'Porto'
data.City.replace('Oporto', 'Porto', inplace=True)

In [ ]:
# составим список из городов, упомянутых в датафрейме
my_cities = list(data.City.value_counts().index)
# из внешних источников загрузим файл с данными о городах мира и составим словари cities_pop город: население и cities_cap город:признак города
# где признак города равен 2 - если это столица, 1 - если это крупный город, 0 - в остальных случаях
worldcities = pd.read_csv('/kaggle/input/world-cities-datasets/worldcities.csv')
worldcities_short = worldcities[['city_ascii', 'capital', 'population']]
cities_pop = {}
cities_cap = {}
for city in my_cities:
    city_list = worldcities_short[worldcities_short.city_ascii==city]
    # в загруженном справочнике есть города с одинаковым названием, выбираем из них один город, где самое большое население
    pop_max = city_list.population.max()
    capital = city_list[city_list.population == pop_max].capital.values[0]
    if capital =='primary':
        cap = 2
    elif capital == 'admin':
        cap = 1
    else:
        cap = 0
    cities_pop[city] = pop_max
    cities_cap[city] = cap

In [ ]:
# посмотрим все ли города попали в словарь
print(f'Количество городов в cities_popt - {len(cities_pop)}')
print(f'Количество городов в my_cities - {len(my_cities)}')

In [ ]:
# создадим колонки с названиям городов
df_cities = pd.get_dummies(data['City'], prefix='City')
data = data.join(df_cities)
display(data.head())

In [ ]:
# в соответствии со словарем заполним столбец признака населения Population
# параллельно заполним признак Capital - признак города
for row in range(data.shape[0]):
    data.loc[row, 'Population'] = cities_pop[data.loc[row, 'City']]
    data.loc[row, 'Capital'] = cities_cap[data.loc[row, 'City']]

data.head()

In [ ]:
# создадим столбец Reviews day, показывающий количество дней межу последними 2 отзывами, если отзывов нет или был всего один, то значение равно 0
import datetime
from datetime import datetime, timedelta
import re


pattern = r'\d{2}/\d{2}/\d{4}'

datetime_list = []
comments_days = []
for ind in range(data.shape[0]):
    dates = re.findall(pattern, str(data.loc[ind,'Reviews']))
    comments_len = len(dates)
    if  comments_len > 0:
        for i in range(comments_len):
            datetime_list.append(datetime.strptime(dates[i], '%m/%d/%Y'))
        if comments_len > 1:
            dist = np.abs(datetime_list[-1] - datetime_list[-2])
            comments_days.append(dist.days)
        else:
            comments_days.append(0)
    else:
        comments_days.append(0)

data['Reviews_days'] = comments_days

In [ ]:
# заменим пропуски в признаке 'Cuisine Style' значением No cuisine
data['Cuisine Style'].fillna(' No cuisine ', inplace=True)

In [ ]:
# создадим словарь с перечнем кухонь и список с количеством кухонь в каждом ресторане, если данных не было, то количество принимаем за 1
cuisine = {}
col_cuisine = []
for ind in range(len(data)):
    s = 0
    string = data.loc[ind, 'Cuisine Style'][1:-1].split(',')
    for item in string:
        item = item.replace("'", '')
        item.strip()
        s += 1
        if ord(item[0]) == 32:
            item = item[1:]
        if item in cuisine:
            cuisine[item] += 1
        else:
            cuisine[item] = 1
    col_cuisine.append(s)

# отсортируем полученный словарь cuisine по убыванию значений
cuisine = {k: v for k, v in sorted(cuisine.items(), key=lambda item: item[1], reverse=True)}

cuisine

In [ ]:
# создадим вспомогательный датафрейм df_cuisine, в котором будут отмечены кухни 10 наиболее популярных кухонь,
# остальные кухни объеденены в 'Others'
# так как значение No cuisine попадает в топ-10 кухонь, то возьмем 11 значений

cuisine_top = list(cuisine)[:11]
df_cuisine = pd.DataFrame(columns=cuisine_top, dtype=int)
for row in range(data.shape[0]):
    counter = 0
    cuisine_string = data.loc[row, 'Cuisine Style']
    for name in cuisine_top:
        if name in cuisine_string:
            df_cuisine.loc[row, name] = 1
            counter += 1
        else:
            df_cuisine.loc[row, name] = 0
    if counter < col_cuisine[row]: 
        df_cuisine.loc[row, 'Others'] = 1
        
# отсавшиеся незаполненными значние в признаке Other заполняем 0
df_cuisine.Others.fillna(0, inplace=True)
df_cuisine.head()

In [ ]:
# добавим этот датафрейм в исходную базу
data = data.join(df_cuisine)
data.head()

In [ ]:
# добавим новый количественный признак Cuisines_sum -  количествj кухонь в ресторане
data['Cuisines_sum'] = col_cuisine

In [ ]:
# исследуем id ресторанов 
data.Restaurant_id.value_counts()

In [ ]:
# судя по тому, что некоторые id повторяются, можно говорить о наличии сетевых ресторанов
# создадим новый признак Chain 
dict_chain = data.Restaurant_id.value_counts()
for id in dict_chain.keys():
    if dict_chain[id] > 1:
        dict_chain[id] = 1
    else:
        dict_chain[id] = 0
for row in range(data.shape[0]):
    data.loc[row, 'Chain'] = dict_chain[data.loc[row, 'Restaurant_id']]

In [ ]:
# напишем функцию по созданию словаря из слов из отобранных отзывов
def words_in_dict(full_string, pattern=r'\d{2}/\d{2}/\d{4}'):
    def_dict = {}
    for row in range(len(full_string)):
        string_rev = str(full_string[row]).split(',')
        if len(string_rev) <= 2:
            continue   
        for phrase in string_rev:
            dates = re.findall(pattern, phrase)
            if len(dates)>0:
                continue
            else:
                words = phrase.split(' ')
                for word in words:
                    word="".join(c for c in word if c.isalpha())
                    word=word.lower()
                    if word in def_dict:
                        def_dict[word] += 1
                    else:
                        def_dict[word] = 1
    def_dict = {k: v for k, v in sorted(def_dict.items(), key=lambda item: item[1], reverse=True)}
    # удаляем из словаря союзы, предлоги, местоимения, мало значащие, но популярные слова
    [def_dict.pop(key) for key in ['', 'a', 'and', 'an', 'the', 'in', 'at', 'for', 'of', 'with', 'it', 'is', 'i', 'as', 'be', 'to', 'food', 'service']]
    return def_dict


# заполним словарь для тех данных, где рейтинг 4 и выше
best_reviews = data[data.Rating>3.5].Reviews
best_reviews.reset_index(drop=True, inplace=True)
good_words = words_in_dict(best_reviews)
print(good_words)

# на основе этих данных создадим вручную список наиболее характерных отзывов для положительных отзывов
yes_words = ['good', 'great', 'nice', 'very', 'excellent', 'lovel', 'amaz', 'delicious', 'atmosphere', 'fantastic', 'tasty']

In [ ]:
# теперь составим аналогичный словарь для отзывов ресторанов с рейтингом от 2.5 и ниже
worst_reviews = data[data.Rating<3].Reviews
worst_reviews.reset_index(drop=True, inplace=True)
bad_words = words_in_dict(worst_reviews)
print(bad_words)

# на основе этих данных создадим вручную список наиболее характерных отзывов для отрицательных отзывов
no_words = ['never', 'no', 'hell', 'bad', 'worst', 'awful', 'terribl', 'sad', 'disappoint', 'not good', 'dont', 'avoid', 'horribl']

In [ ]:
# на основе полученных списков заполняем новые признаки Bad reviews и Good reviews
# если "плохое" слово присутствует в отзыве, то ставим 1 у Bad reviews
# причем, если в отзыве текущей строки есть слова из списка "плохих", то Good reviews = 0
data['Good reviews'] = 0
data['Bad reviews'] = 0
data.Reviews.fillna('Empty', inplace=True)
for row in range(data.shape[0]):
    for word_n in no_words:
        if word_n in data.loc[row, 'Reviews'].lower():
            data.loc[row, 'Bad reviews'] = 1
    if data.loc[row, 'Bad reviews'] == 0:
        for word_y in yes_words:
            if word_y in data.loc[row, 'Reviews'].lower():
                data.loc[row, 'Good reviews'] = 1

data.head()

#### Возьмем следующий признак "Price Range".

In [ ]:
# всего в признаке 'Price Range' выделяются 3 состояния
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# заменим в столбце 'Price Range' символьные значения на более понятные: $$$$ - 3, $$ - $$$ - 2, $ - 1, пропуски заполним отличным значением - 0
dict_price = {'$$$$':3, '$$ - $$$':2,'$':1}
data.replace({'Price Range':dict_price}, inplace=True)
data['Price Range'].fillna(0, inplace=True)
data['Price Range']

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями
worldcities = pd.read_csv('/kaggle/input/world-cities-datasets/worldcities.csv')
worldcities_short = worldcities[['city_ascii', 'capital', 'population']]

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
        
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['URL_TA','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['NumberReviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Price Range'].fillna(0, inplace=True)
    df_output['Cuisine Style'].fillna(' No cuisine ', inplace=True)
    df_output['Reviews'].fillna('Empty', inplace=True)
    
    
    # ################### 3. Encoding ############################################################## 
    # заменим в столбце 'Price Range' символьные значения на более понятные: $$$$ - 3, $$ - $$$ - 2, $ - 1, пропуски заполним отличным значением - 0
    dict_price = {'$$$$':3, '$$ - $$$':2,'$':1}
    df_output.replace({'Price Range':dict_price}, inplace=True)
    # изменим название одного из городов 'Oporto' на более используемое в справочниках 'Porto'
    df_output.City.replace('Oporto', 'Porto', inplace=True)
    # составим список из городов, упомянутых в датафрейме
    my_cities = list(df_output.City.value_counts().index)
    cities_pop = {}
    cities_cap = {}
    for city in my_cities:
        city_list = worldcities_short[worldcities_short.city_ascii==city]
        # в загруженном справочнике есть города с одинаковым названием, выбираем из них один город, где самое большое население
        pop_max = city_list.population.max()
        capital = city_list[city_list.population == pop_max].capital.values[0]
        if capital =='primary':
            cap = 2
        elif capital == 'admin':
            cap = 1
        else:
            cap = 0
        cities_pop[city] = pop_max
        cities_cap[city] = cap
    
    # в соответствии со словарем заполним столбец признака населения Population
    # параллельно заполним признак Capital - признак города
    for row in range(df_output.shape[0]):
        df_output.loc[row, 'Population'] = cities_pop[df_output.loc[row, 'City']]
        df_output.loc[row, 'Capital'] = cities_cap[df_output.loc[row, 'City']]
    # создадим колонки с названиям городов
    df_cities = pd.get_dummies(df_output['City'], prefix='City')
    df_output = df_output.join(df_cities)
    
    # ################### 4. Feature Engineering ####################################################
    # создадим столбец Reviews day, показывающий количество дней межу последними 2 отзывами, если отзывов нет или был всего один, то значение равно 0
    pattern = r'\d{2}/\d{2}/\d{4}'

    datetime_list = []
    comments_days = []
    for ind in range(df_output.shape[0]):
        dates = re.findall(pattern, str(df_output.loc[ind,'Reviews']))
        comments_len = len(dates)
        if  comments_len > 0:
            for i in range(comments_len):
                datetime_list.append(datetime.strptime(dates[i], '%m/%d/%Y'))
            if comments_len > 1:
                dist = np.abs(datetime_list[-1] - datetime_list[-2])
                comments_days.append(dist.days)
            else:
                comments_days.append(0)
        else:
            comments_days.append(0)

    df_output['Reviews_days'] = comments_days
    
    # создадим словарь с перечнем кухонь и список с количеством кухонь в каждом ресторане, если данных не было, то количество принимаем за 1
    cuisine = {}
    col_cuisine = []
    for ind in range(len(df_output)):
        s = 0
        string = df_output.loc[ind, 'Cuisine Style'][1:-1].split(',')
        for item in string:
            item = item.replace("'", '')
            item.strip()
            s += 1
            if ord(item[0]) == 32:
                item = item[1:]
            if item in cuisine:
                cuisine[item] += 1
            else:
                cuisine[item] = 1
        col_cuisine.append(s)
    # отсортируем полученный словарь cuisine по убыванию значений
    cuisine = {k: v for k, v in sorted(cuisine.items(), key=lambda item: item[1], reverse=True)}
    
    # создадим вспомогательный датафрейм df_cuisine, в котором будут отмечены кухни 10 наиболее популярных кухонь,
    # остальные кухни объеденены в 'Others'
    # так как значение No cuisine попадает в топ-10 кухонь, то возьмем 11 значений
    cuisine_top = list(cuisine)[:11]
    df_cuisine = pd.DataFrame(columns=cuisine_top, dtype=int)
    for row in range(df_output.shape[0]):
        counter = 0
        cuisine_string = df_output.loc[row, 'Cuisine Style']
        for name in cuisine_top:
            if name in cuisine_string:
                df_cuisine.loc[row, name] = 1
                counter += 1
            else:
                df_cuisine.loc[row, name] = 0
        if counter < col_cuisine[row]: 
            df_cuisine.loc[row, 'Others'] = 1
        
    # отсавшиеся незаполненными значние в признаке Other заполняем 0
    df_cuisine.Others.fillna(0, inplace=True)
    # добавим этот датафрейм в исходную базу
    df_output = df_output.join(df_cuisine)
    
    # добавим новый количественный признак Cuisines_sum - количество кухонь в ресторане 
    df_output['Cuisines_sum'] = col_cuisine
    
    # создадим новый признак Chain 
    dict_chain = df_output.Restaurant_id.value_counts()
    for id in dict_chain.keys():
        if dict_chain[id] > 1:
            dict_chain[id] = 1
        else:
            dict_chain[id] = 0
    for row in range(df_output.shape[0]):
        df_output.loc[row, 'Chain'] = dict_chain[df_output.loc[row, 'Restaurant_id']]
        
    yes_words = ['good', 'great', 'nice', 'very', 'excellent', 'lovel', 'amaz', 'delicious', 'atmosphere', 'fantastic', 'tasty']
    no_words = ['never', 'no', 'hell', 'bad', 'worst', 'awful', 'terribl', 'sad', 'disappoint', 'not good', 'dont', 'avoid', 'horribl']
    # на основе полученных списков заполняем новые признаки Bad reviews и Good reviews
    # если "плохое" слово присутствует в отзыве, то ставим 1 у Bad reviews
    # причем, если в отзыве текущей строки есть слова из списка "плохих", то Good reviews = 0
    df_output['Good reviews'] = 0
    df_output['Bad reviews'] = 0
    for row in range(df_output.shape[0]):
        for word_n in no_words:
            if word_n in df_output.loc[row, 'Reviews'].lower():
                df_output.loc[row, 'Bad reviews'] = 1
        if df_output.loc[row, 'Bad reviews'] == 0:
            for word_y in yes_words:
                if word_y in df_output.loc[row, 'Reviews'].lower():
                    df_output.loc[row, 'Good reviews'] = 1
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# округлим значения y_pred до 0.5
def round_to_05(x):
    return np.round(x*2)/2


y_pred = round_to_05(y_pred)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission = round_to_05(predict_submission)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

Выводы: 
1. Исследуемый датасет слабо реагировал на добавление новых признаков изменением метрики MAE до того самого момента, пока не вставил код, округляющий значения y_predict до 0.5. Это сдвинуло значение метрики с 0,21 на 0,18-0,17.
2. За прошедшее время с начала обучения я НЕ научился приемам "убыстрения" обработки базы. Полный прогон кода занимает порядка 20 минут, что, на мой взгляд, очень долго. (Хотелось бы, чтобы этим приемам именно УЧИЛИ на курсе, а не мы сами приходили к этому, шертся сотни страниц google). Пока в этом моменте курсы меня разочаровывают.
3. В целом, было создано и оттестировано несколько новых количественных признаков, в том числе, с задействованием внешних источников информации.
4. Были реализованы разные подходы к обработке пропусков - заполнение наиболее часто встречающимся значением, заполнение отличным от других значением (чтобы не создавать отдельный признак отсутствия информации), заполнение константой.
5. Были задействованы знания по использованию библиотек re и datetime.
6. Были использованы приемы преобразования категориальных признаков в количественные с помощью метода get_dummies.
7. В этом задании очень много работал с методом .loc, но не могу дать оценку насколько правильно я его задействовал, так как в целом меня не удовлетворили результаты скорости обработки информации моим кодом (см. п. 2).
8. Довольно часто создавал словари, мне нравится их создавать. Но вопрос траты времени на создание словарей по сравнению с другими подходами остается открытым.
9. В целом оцениваю уровень своего креатива на 3 по 5-балльной системе. 
10. Задание далось неожиданно очень тяжело. Сильно его усложняла скорость обработки (см. п. 2), так как каждый переапуск кода занимал очень долгое время.
